# Préambule pour Colab

Tout d'abord, sélectionnez l'option GPU de Colab avec *Edit > Notebook settings* et sélectionner GPU comme Hardware accelerator. Installer ensuite deeplib avec la comamnde suivante:

In [ ]:
!pip install git+https://github.com/ulaval-damas/glo4030-labs.git

# Laboratoire 5: CNN

In [ ]:
from deeplib.training import train, test
import torch.optim as optim
import torch
import numpy as np
from deeplib.datasets import load_cifar10, load_mnist
from deeplib.visualization import view_filters
import torch.nn.functional as F
import torch.nn as nn
from random import randrange

cifar_train, cifar_test = load_cifar10(download=True)
mnist_train, mnist_test = load_mnist(download=True)

## Filtres de convolution

Lors de l'entraînement, le réseau apprend les bons paramètres à utiliser. Par contre, autrefois, il fallait utiliser des filtres fait à la main comme [les filtres de Gabor](https://en.wikipedia.org/wiki/Gabor_filter).

### Exercice

Le réseau suivant contient une seule couche de convolution. 

Créez manuellement quelques fitres que vous utiliserez pour faire de la classification sur CIFAR10. 

Par la suite, figez les poids de la couche de convolution et entraînez le réseau. 
Tentez d'obtenir les meilleurs résultat possible.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, padding=1)
        self.fc = nn.Linear(6 * 14 * 14, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = x.view(-1, 6*14*14)
        x = self.fc(x)
        return x

Modifier les paramètres des filtres. Essayez de faire des filtres permettant d'extraire des caractéristiques de bas niveau (ligne, coin, etc...). Vous pouvez consulter [ceci](http://lodev.org/cgtutor/filtering.html) pour avoir des idées.

In [ ]:
filters = []
filters.append([[[0, 0, 0],
                 [0, 1, 0],
                 [0, 0, 0]]]) # Ce filtre retourne l'image original

filters.append([[[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, 0]]])

filters.append([[[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, 0]]])

filters.append([[[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, 0]]])

filters.append([[[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, 0]]])

filters.append([[[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, 0]]])

In [ ]:
# On crée le réseau, remplace les paramètres par les filtres précédents et fige les poids.

net = Net()
filters = np.asarray(filters, dtype=np.float32)
net.conv1.weight.data = torch.from_numpy(filters)
for param in net.conv1.parameters():
    param.requires_grad = False
net = net.cuda()

In [ ]:
# Vous pouvez utiliser cette cellule pour visualiser l'effet de vos filtres sur des images du dataset.
for i in range(3):
    image, label = mnist_train[randrange(0, len(mnist_train))]
    view_filters(net, image)

In [ ]:
lr = 0.001
n_epoch = 5
batch_size = 32

In [ ]:
optimizer = optim.Adam(net.fc.parameters(), lr=lr) # On optimise uniquement la couche pleinement connectée.
history = train(net, optimizer, mnist_train, n_epoch, batch_size)
history.display_accuracy()
history.display_loss()
print('Précision en test: {:.2f}'.format(test(net, mnist_test, batch_size)))

## Architecture de base

### Exercice

Implémentez une architecture de base de réseau de neurones à convolution ayant les caractéristiques suivantes.

1. 3 couches de convolution
2. Toutes les couches ont 100 filtres de tailles 3x3 et 1px de padding.
3. Batch normalization après chaque couche.
4. Maxpooling avec un noyau de taille 2 après les 2 premières couches.
5. 1 seule couche linéaire pour la classification (aucune activation nécessaire)
6. Utiliser la ReLu comme fonction d'activation.

Notez que la taille des images de CIFAR10 est de 3x32x32 (images en couleur). 

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
lr = 0.01
n_epoch = 10
batch_size = 32

In [ ]:
model = ConvNet()
model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr)
history = train(model, optimizer, cifar_train, n_epoch, batch_size)
history.display_accuracy()
history.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))

## Architecture profonde

En deep learning, on dit que la performance augmente avec le nombre de couches.

### Exercice

Ajoutez 2 couches de convolution de 100 filtres dans le réseau précédent (n'oubliez pas la batch normalization et le padding). Mettez du maxpooling après la couche 1 et 3. Comparez les résultats.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
model = ConvNet()
model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr)
history = train(model, optimizer, cifar_train, n_epoch, batch_size)
history.display_accuracy()
history.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))

### Exercice

Ajoutez 4 autres couches de 100 filtres avec batchnorm et padding de 1. Mettez le maxpooling après les couches 3 et 5.

Est-ce que les résultats continuent à s'améliorer?

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
model = ConvNet()
model.cuda()

lr = 0.0005
optimizer = optim.Adam(model.parameters(), lr=lr)
history = train(model, optimizer, cifar_train, n_epoch, batch_size)
history.display_accuracy()
history.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))

## Connexion résiduelle

Ajouter de plus en plus de couche augmente aussi la difficulté avec laquelle le gradient peut se propager dans le réseau. Une des solutions suggérés est d'utiliser une connexion résiduelle permettant au gradient de _sauter_ des couches. Dans l'article présentant cette connexion, elle est définie comme suit:

![alt text](http://cv-tricks.com/wp-content/uploads/2017/03/600x299xResNet.png.pagespeed.ic.ZVwbFN7vyG.webp "Connexion résiduelle")

### Exercice
Reprenez l'architecture précédente et ajouter des connexion résiduelle à chaque 2 couches en commençant à la couche 2. Faites un maxpool après la connexion résiduelle suivant la couche 3 et 5.

Comparez les résultats et la vitesse avec laquelle le réseau a entraîné.

In [ ]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
model = Resnet()
model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr)
history = train(model, optimizer, cifar_train, n_epoch, batch_size)
history.display_accuracy()
history.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))